In [2]:
%cd /scratch/mt/ashapiro/Hate_Speech/
from utils import getData
from datasets import load_dataset 
import pandas as pd 
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd 
import numpy as np
import torch
import matplotlib.pyplot as plt
from collections import Counter
from datasets import load_metric
from transformers import EarlyStoppingCallback

/scratch/mt/ashapiro/Hate_Speech


In [3]:
f1 = load_metric("f1")
recall = load_metric("recall")
precision =  load_metric("precision")
def preprocess_function(examples, tok):
    return tok(examples["text"], truncation=True, max_length=512)


def compute_metrics(p):    
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    metric = f1.compute(predictions=predictions, references=labels, average="macro")
    metric.update(recall.compute(predictions=predictions, references=labels, average="macro"))
    metric.update(precision.compute(predictions=predictions, references=labels, average="macro"))
    return metric
data = getData(sub_task = f"A", return_type = "dataset", pre_proccessed = True)
tokenizer = AutoTokenizer.from_pretrained("/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_data = data.map(preprocess_function,fn_kwargs = {'tok':tokenizer}, batched=True)

Pre-Processed


100%|██████████| 2/2 [00:00<00:00, 29.27ba/s]


BestRun(run_id='21', objective=2.572288460648268, hyperparameters={'learning_rate': 2.7589028213603294e-05, 'weight_decay': 1.496202107421057e-05, 'num_train_epochs': 1, 'seed': 20, 'per_device_train_batch_size': 32})

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2", num_labels=2)
training_args = TrainingArguments(
    output_dir="/scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers",
    learning_rate=2.7589028213603294e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    metric_for_best_model="f1",
    num_train_epochs=100,
    weight_decay=1.496202107421057e-05,
    load_best_model_at_end=True,
    group_by_length = True, 
    seed = 20,
    report_to = None
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

loading configuration file /scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2/config.json
Model config BertConfig {
  "_name_or_path": "/scratch/mt/ashapiro/Hate_Speech/Models/Marbertv2",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading weights file /scra

In [9]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 8887
  Num Epochs = 100
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 27800
  1%|          | 277/27800 [00:27<37:18, 12.29it/s]  The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4

 97%|█████████▋| 309/318 [00:04<00:00, 75.77it/s]
                                                   A
100

{'eval_loss': 0.32315635681152344, 'eval_f1': 0.8505582936759521, 'eval_recall': 0.8512536299819359, 'eval_precision': 0.8498765804675422, 'eval_runtime': 4.2469, 'eval_samples_per_second': 299.041, 'eval_steps_per_second': 74.878, 'epoch': 1.0}


Model weights saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-278/pytorch_model.bin
tokenizer config file saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-278/tokenizer_config.json
Special tokens file saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-278/special_tokens_map.json
  2%|▏         | 501/27800 [00:58<40:38, 11.19it/s]  

{'loss': 0.3382, 'learning_rate': 2.7092822670193162e-05, 'epoch': 1.8}


  2%|▏         | 555/27800 [01:03<39:43, 11.43it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4

 97%|█████████▋| 308/318 [00:04<00:00, 73.74it/s]
                                                   A
100%|██████████| 318/318 [00:04<00:00, 74.21it/s]
                                                 Saving model checkpoint to /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-556
Configuration saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-556/config.json


...

 11%|█         | 3058/27800 [06:25<49:34,  8.32it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1270
  Batch size = 4

 96%|█████████▌| 305/318 [00:03<00:00, 78.16it/s]
                                                    
100%|██████████| 318/318 [00:04<00:00, 78.34it/s]
                                                 Saving model checkpoint to /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-3058
Configuration saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-3058/config.json


{'eval_loss': 1.0896872282028198, 'eval_f1': 0.8309000617462785, 'eval_recall': 0.8236114604532047, 'eval_precision': 0.8400000000000001, 'eval_runtime': 4.1986, 'eval_samples_per_second': 302.481, 'eval_steps_per_second': 75.739, 'epoch': 11.0}


Model weights saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-3058/pytorch_model.bin
tokenizer config file saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-3058/tokenizer_config.json
Special tokens file saved in /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-3058/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /scratch/mt/ashapiro/Hate_Speech/Checkpoints/SentenceBert/Classifiers/checkpoint-278 (score: 0.8505582936759521).
 11%|█         | 3058/27800 [06:33<53:01,  7.78it/s]

{'train_runtime': 393.2281, 'train_samples_per_second': 2260.011, 'train_steps_per_second': 70.697, 'train_loss': 0.10183306980787966, 'epoch': 11.0}


TrainOutput(global_step=3058, training_loss=0.10183306980787966, metrics={'train_runtime': 393.2281, 'train_samples_per_second': 2260.011, 'train_steps_per_second': 70.697, 'train_loss': 0.10183306980787966, 'epoch': 11.0})